### Imports

In [39]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import os
import sklearn
import gc
import time
import timm
import random
import safetensors
from safetensors.torch import load_file
from PIL import Image
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from torchvision import transforms
from torchvision import models
from collections import defaultdict
from sklearn.model_selection import GroupKFold
from torch.utils.data import random_split
from scipy.stats import zscore
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') != 'Batch':
    !pip install -q ipdb
    import ipdb

In [40]:
SET_SEED=42
def set_seed(seed=SET_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed

<function __main__.set_seed(seed=42)>

In [41]:
# Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 8
NUM_FT_EPOCHS = 20
NUM_BB_EPOCHS = 12
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1e-7
NUM_FOLDS = 3
GIVEN_WEIGHTS = [0.1, 0.1, 0.1, 0.5, 0.2]
TARGET_COLS = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
BASE_MODEL='resnet50'
IMAGE_SIZE=(384,384)

In [42]:
def print_result(train_loss, val_loss, epoch_start, epoch, num_epochs, val_r2):
    epoch_time = time.time() - epoch_start
    mins, secs = divmod(epoch_time, 60)
    
    print(f'Epoch {epoch+1}/{num_epochs} - '
          f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | '
          f'R²: {val_r2:.4f} | '
          f'Time: {int(mins)}m {int(secs)}s')

class BiomassDataset(torch.utils.data.Dataset):
    def __init__(self, df, base_path, transform=None):
        self.df = df
        self.base_path = base_path
        self.transform = transform
        self.target_cols = TARGET_COLS
        self.is_training = all(col in df.columns for col in self.target_cols)
    
    def __len__(self):
        return len(self.df) * 2
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx // 2] 
        img_path = os.path.join(self.base_path, row['image_path'])
        
        image = Image.open(img_path).convert('RGB')
        
        half = idx % 2
        width, height = image.size
        if half == 0:
            image = image.crop((0, 0, width // 2, height))  # Left half
        else:
            image = image.crop((width // 2, 0, width, height))  # Right half
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_training:
            targets = row[self.target_cols].values.astype('float32')
            
            return image, torch.tensor(targets, dtype=torch.float32)
        else:            
            return image, row['image_path']

class ExtraDataset(torch.utils.data.Dataset):
    def __init__(self, df, img_path, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_path = img_path
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_file = os.path.join(self.img_path, row['image_file_name'])
        image = Image.open(img_file).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        target = torch.tensor(row['dry_total'], dtype=torch.float32)
        return image, target

In [43]:
class PreTrainModel(nn.Module):
    def __init__(self):
        super().__init__()
        model = timm.create_model(BASE_MODEL, pretrained=False, num_classes=0)
        
        ckpt_path = "/kaggle/input/m/voxoff/resnet50/pytorch/default/1/model.safetensors"
        # model.load_state_dict(load_file(ckpt_path))


        loaded_state_dict = load_file(ckpt_path)

        new_state_dict = {}
        for k, v in loaded_state_dict.items():
            # Assuming the actual ResNet backbone weights are nested under 'resnet.encoder.'
            if k.startswith('resnet.encoder.'):
                new_key = k[len('resnet.encoder.'):] # Strip the prefix
                new_state_dict[new_key] = v

        # Load the modified state dict, allowing for non-matching keys (strict=False)
        model.load_state_dict(new_state_dict, strict=False)

        
        self.backbone = model
        in_features = self.backbone.num_features
        
        self.regression_head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.regression_head(features)

In [44]:
class FinetuneModel(nn.Module):
    def __init__(self, pretrained_backbone):
        super().__init__()
        self.backbone = pretrained_backbone
        in_features = self.backbone.num_features
        
        self.regression_head = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 5) # 5 outputs for competition
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.regression_head(features)

In [56]:
base = '/kaggle/input'
train_csv = f'{base}/csiro-biomass/train.csv'
test_csv = f'{base}/csiro-biomass/test.csv'
extra_csv = f'{base}/grassclover-dataset/biomass_data/train/biomass_train_data.csv'
extra_img = f'{base}/grassclover-dataset/biomass_data/train/images'
base_path = f'{base}/csiro-biomass/'
submission_path = f'{base}/csiro-biomass/sample_submission.csv'

dataset_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
extra_df = pd.read_csv(extra_csv, sep=';')
extra_img_path = extra_img
unique_test_images = test_df['image_path'].unique()

In [57]:
dataset_df['Sampling_Date'] = pd.to_datetime(dataset_df['Sampling_Date'], format='mixed')  # adjust format if needed
dataset_df = dataset_df.pivot(
    index=['image_path','Sampling_Date'],
    columns='target_name',
    values='target'
).reset_index()
dataset_df['Month'] = dataset_df['Sampling_Date'].dt.month

Dry_Clover_g: mean=6.65, std=12.12
Dry_Dead_g: mean=12.04, std=12.40
Dry_Green_g: mean=26.62, std=25.40
Dry_Total_g: mean=45.32, std=27.98
GDM_g: mean=33.27, std=24.94


target_name,image_path,Sampling_Date,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g,Month
0,train/ID1011485656.jpg,2015-09-04,0.0000,31.9984,16.2751,48.2735,16.2750,9
1,train/ID1012260530.jpg,2015-04-01,0.0000,0.0000,7.6000,7.6000,7.6000,4
2,train/ID1025234388.jpg,2015-09-01,6.0500,0.0000,0.0000,6.0500,6.0500,9
3,train/ID1028611175.jpg,2015-05-18,0.0000,30.9703,24.2376,55.2079,24.2376,5
4,train/ID1035947949.jpg,2015-09-11,0.4343,23.2239,10.5261,34.1844,10.9605,9


In [ ]:
# def denormalize_targets(normalized_targets):
#     """
#     Convert normalized targets back to original scale.
    
#     Args:
#         normalized_targets: Tensor of shape [batch_size, 5] or [5] with normalized values
    
#     Returns:
#         Tensor of same shape with denormalized values
#     """
#     if normalized_targets.dim() == 1:
#         # Single sample: [5]
#         means = TARGET_MEANS.to(normalized_targets.device)
#         stds = TARGET_STDS.to(normalized_targets.device)
#         return normalized_targets * stds + means
#     else:
#         # Batch: [batch_size, 5]
#         means = TARGET_MEANS.to(normalized_targets.device).unsqueeze(0)  # [1, 5]
#         stds = TARGET_STDS.to(normalized_targets.device).unsqueeze(0)  # [1, 5]
#         return normalized_targets * stds + means


# # Normalization
# target_stats = {}
# for col in TARGET_COLS:
#     target_stats[col] = {
#         'mean': dataset_df[col].mean(),
#         'std': dataset_df[col].std() + 1e-8
#     }
#     print(f"{col}: mean={target_stats[col]['mean']:.2f}, std={target_stats[col]['std']:.2f}")

# # Store for later denormalization
# TARGET_MEANS = torch.tensor([target_stats[col]['mean'] for col in TARGET_COLS], dtype=torch.float32)
# TARGET_STDS = torch.tensor([target_stats[col]['std'] for col in TARGET_COLS], dtype=torch.float32)

# dataset_df.head()

### Pytorch

In [47]:
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomRotation(15),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

### Support Functions

In [48]:
def forward_pass(images, targets, optimizer, model, validation=False):
    images = images.to(device)
    targets = targets.to(device)
    
    if not validation: 
        optimizer.zero_grad()
    
    outputs = model(images)
    
    loss = combined_biomass_loss(outputs, targets)
    
    if not validation:
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    
    return loss

### Train Model

In [49]:
def combined_biomass_loss(biomass_pred, biomass_true):
    weights = torch.tensor(GIVEN_WEIGHTS, device=biomass_pred.device)

    smooth_l1 = nn.SmoothL1Loss(reduction='none')
    mse = nn.MSELoss(reduction='none')
    
    smooth_l1_loss = smooth_l1(biomass_pred, biomass_true)
    mse_loss = mse(biomass_pred, biomass_true)
    
    combined = 0.3 * smooth_l1_loss + 0.7 * mse_loss
    weighted_loss = (combined * weights).mean()
    
    return weighted_loss

In [50]:
def weighted_r2_score(sum_target, total_samples, sum_target_sq, ss_res):
    mean_target = sum_target / total_samples
    ss_tot = sum_target_sq - total_samples * (mean_target ** 2)

    r2_per_output = 1 - ss_res / (ss_tot + 1e-10)

    weights = torch.tensor(GIVEN_WEIGHTS, device=device)
    r2_weighted = (r2_per_output * weights).sum() / weights.sum()
    return r2_weighted

In [51]:
def pretrain_phase(extra_df, extra_img_path, num_epochs=NUM_FT_EPOCHS):
    print("=" * 50)
    print("PHASE 1: PRE-TRAINING ON EXTRA DATASET")
    
    # Create dataset
    extra_dataset = ExtraDataset(extra_df, extra_img_path, transform=train_transform)

    generator = torch.Generator().manual_seed(SET_SEED)
    
    extra_train_size = int(0.8 * len(extra_dataset))
    extra_dev_size = len(extra_dataset) - extra_train_size

    extra_train_dataset, extra_dev_dataset = random_split(extra_dataset, [extra_train_size, extra_dev_size], generator=generator)

    extra_loader = DataLoader(extra_train_dataset, batch_size=16, shuffle=True)
    extra_dev_loader = DataLoader(extra_dev_dataset, batch_size=16, shuffle=False)  # usually no shuffle for dev
    
    # Initialize model
    model = PreTrainModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.SmoothL1Loss()
    
    gc.collect()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for images, targets in extra_loader:
            images, targets = images.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
            # validate_epoch(model, extra_dev_loader, criterion=criterion)
        
        print(f"Pre-train Epoch {epoch+1}/{num_epochs} - Loss: {total_loss/len(extra_loader):.4f}")
    
    return model.backbone 

In [52]:
def create_data_loaders(train_df, val_df, batch_size=BATCH_SIZE):
    train_dataset = BiomassDataset(train_df, base_path, transform=train_transform)
    val_dataset = BiomassDataset(val_df, base_path, transform=val_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return train_loader, val_loader

def train_epoch(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for images, targets in train_loader:
        loss = forward_pass(images, targets, optimizer, model)
        total_loss += loss
    return total_loss / len(train_loader)

def validate_epoch(model, val_loader, criterion=None):
    model.eval()
    total_loss = 0
    total_samples = 0

    # Accumulate sums for R² calculation (no need to store all predictions)
    ss_res = torch.zeros(5, device=device)
    sum_target = torch.zeros(5, device=device)
    sum_target_sq = torch.zeros(5, device=device)

    with torch.no_grad():
        for images, targets in val_loader:
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)

            if criterion:
                loss = criterion(outputs, targets)
            else:
                loss = combined_biomass_loss(outputs, targets)
            
            total_loss += loss.item()
            total_samples += outputs.shape[0] # batch_size
            
            ss_res += ((outputs - targets) ** 2).sum(dim=0)
            sum_target += targets.sum(dim=0)
            sum_target_sq += (targets ** 2).sum(dim=0)

    r2_weighted = weighted_r2_score(sum_target, total_samples, sum_target_sq, ss_res)

    return total_loss / len(val_loader), r2_weighted

def finetune_phase(pretrained_backbone, num_epochs=NUM_FT_EPOCHS):
    print("\n" + "=" * 50)
    print("PHASE 2: FINE-TUNING ON COMPETITION DATA")
    
    r2 = []

    for fold, (train_idx, val_idx) in enumerate(splits):
        print(f'\n--- Fold {fold + 1}/{NUM_FOLDS} ---')
        
        train_df = dataset_df.iloc[train_idx].copy()
        val_df = dataset_df.iloc[val_idx].copy()
        train_loader, val_loader = create_data_loaders(train_df, val_df)

        model = FinetuneModel(copy.deepcopy(pretrained_backbone)).to(device)
        for param in model.backbone.parameters():
            param.requires_grad = False
            
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        
        for epoch in range(num_epochs):
            epoch_start = time.time()
            train_loss = train_epoch(model, train_loader, optimizer)
            val_loss, val_r2 = validate_epoch(model, val_loader)
            
            print_result(train_loss, val_loss, epoch_start, epoch, num_epochs, val_r2)

        r2.append(val_r2.item())
        fold_models.append(model)
        print(f'Fold {fold + 1} complete!')

    overall_r2 = np.array(r2).mean()
    
    print(f'\nOverall R² across all folds: {overall_r2:.4f}')

    return fold_models

In [53]:
pretrained_backbone = pretrain_phase(extra_df, extra_img_path, num_epochs=NUM_BB_EPOCHS)

PHASE 1: PRE-TRAINING ON EXTRA DATASET
Pre-train Epoch 1/12 - Loss: 65.4099
Pre-train Epoch 2/12 - Loss: 64.2731
Pre-train Epoch 3/12 - Loss: 62.7436
Pre-train Epoch 4/12 - Loss: 61.0524
Pre-train Epoch 5/12 - Loss: 58.8126
Pre-train Epoch 6/12 - Loss: 56.2219
Pre-train Epoch 7/12 - Loss: 53.2763
Pre-train Epoch 8/12 - Loss: 49.6969
Pre-train Epoch 9/12 - Loss: 45.2294
Pre-train Epoch 10/12 - Loss: 40.0301
Pre-train Epoch 11/12 - Loss: 34.7882
Pre-train Epoch 12/12 - Loss: 30.1635


In [54]:
kfold = GroupKFold(n_splits=NUM_FOLDS)
groups = dataset_df['Month']
splits = kfold.split(dataset_df, groups=groups)
fold_models = []
final_model = finetune_phase(pretrained_backbone, num_epochs=NUM_FT_EPOCHS)


PHASE 2: FINE-TUNING ON COMPETITION DATA

--- Fold 1/3 ---
Epoch 1/20 - Train Loss: 250.2351, Val Loss: 237.0026 | R²: -1.7977 | Time: 0m 44s
Epoch 2/20 - Train Loss: 173.9674, Val Loss: 108.6896 | R²: -0.4166 | Time: 0m 44s
Epoch 3/20 - Train Loss: 92.2561, Val Loss: 61.4008 | R²: 0.1656 | Time: 0m 44s
Epoch 4/20 - Train Loss: 80.9140, Val Loss: 75.0408 | R²: 0.0198 | Time: 0m 44s
Epoch 5/20 - Train Loss: 73.6500, Val Loss: 78.8854 | R²: -0.0254 | Time: 0m 44s
Epoch 6/20 - Train Loss: 70.5038, Val Loss: 90.1469 | R²: -0.1502 | Time: 0m 44s
Epoch 7/20 - Train Loss: 70.0943, Val Loss: 87.7616 | R²: -0.1208 | Time: 0m 44s
Epoch 8/20 - Train Loss: 61.0279, Val Loss: 108.9406 | R²: -0.3454 | Time: 0m 44s
Epoch 9/20 - Train Loss: 64.8468, Val Loss: 88.0971 | R²: -0.0971 | Time: 0m 44s
Epoch 10/20 - Train Loss: 60.8941, Val Loss: 81.8055 | R²: -0.0585 | Time: 0m 44s
Epoch 11/20 - Train Loss: 57.7155, Val Loss: 81.0488 | R²: -0.0317 | Time: 0m 44s
Epoch 12/20 - Train Loss: 54.0704, Val Loss:

KeyboardInterrupt: 

In [ ]:
test_dataset = BiomassDataset(test_df, base_path, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

ensemble_outputs = []
with torch.no_grad():
    for image, img_path in test_loader:
        image = image.to(device)
        
        # Get predictions from all models
        all_outputs = []
        for model in fold_models:
            model.eval()
            outputs = model(image)
            print(outputs)
            all_outputs.append(outputs)
            ensemble_outputs.append(torch.stack(all_outputs).mean(dim=0))
        
ensemble_outputs = torch.cat(ensemble_outputs, dim=0)
2

In [ ]:
test_dataset = BiomassDataset(test_df, base_path, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Dictionary to store predictions grouped by image_path
preds_by_image = defaultdict(list)

with torch.no_grad():
    for image, img_paths in test_loader:
        image = image.to(device)
        
        # Get ensemble predictions from all models
        all_outputs = []
        for model in fold_models:
            model.eval()
            outputs = model(image)
            all_outputs.append(outputs)
        
        # Average across models (ensemble)
        ensemble_batch = torch.stack(all_outputs).mean(dim=0)  # [batch_size, 5]
        
        # Group predictions by image_path
        # Since BiomassDataset returns 2 items per image (left/right halves),
        # img_paths is a tuple/list of image paths (may have duplicates)
        # We group by image_path and will average left/right later
        batch_size = ensemble_batch.shape[0]
        for i in range(batch_size):
            img_path = img_paths[i]  # Get the image_path for this batch item
            preds_by_image[img_path].append(ensemble_batch[i].cpu())

# Average left/right predictions for each image
image_predictions = {}
for img_path, preds in preds_by_image.items():
    # Stack and average: [num_halves, 5] -> [5]
    # Each image should have exactly 2 predictions (left + right)
    stacked_preds = torch.stack(preds)  # [2, 5] for left and right halves
    image_predictions[img_path] = stacked_preds.mean(dim=0)  # Average to [5]ee

# Check predictions for first image
if len(image_predictions) > 0:
    first_img_path = list(image_predictions.keys())[0]
    print(f"Image: {first_img_path}")
    print(f"Predictions: {image_predictions[first_img_path].tolist()}")
    print(f"Target columns: {TARGET_COLS}")
else:
    print("No predictions available yet. Run the test prediction cell first.")

In [ ]:
ensemble_outputs[0].tolist()

In [ ]:
submission_df = pd.read_csv(submission_path)

for i, row in submission_df.iterrows():
    img_path = test_df[test_df['sample_id'] == row['sample_id']]['image_path'].values[0]
    target_name = test_df[test_df['sample_id'] == row['sample_id']]['target_name'].values[0]
    
    preds = ensemble_outputs
    target_idx = TARGET_COLS.index(target_name)
    submission_df.at[i, 'target'] = ensemble_outputs[i, target_idx].item()

submission_df.to_csv('submission.csv', index=False)
submission_df